In [1]:
from os import listdir
import glob
import pandas as pd
import numpy as np
from datetime import datetime
from statsmodels.tsa.arima_model import ARIMA, ARIMAResults
from pmdarima import auto_arima
from pptx import Presentation 
from pptx.chart.data import CategoryChartData
from pptx.dml.color import RGBColor
from pptx.enum.chart import XL_CHART_TYPE, XL_LEGEND_POSITION, XL_DATA_LABEL_POSITION, XL_MARKER_STYLE, XL_TICK_MARK
from pptx.enum.dml import MSO_LINE_DASH_STYLE, MSO_THEME_COLOR
from pptx.util import Inches, Pt

In [2]:
def get_actual_data(med, nm = "events", dt = "CLINICAL_EVENT_DATETIME"):
    filepaths = glob.glob("../data/tidy/" + med + "/*_" + nm + "_*.csv")
    df = pd.concat(map(pd.read_csv, filepaths), sort=False)
    df["CLINICAL_EVENT_DATETIME"] = pd.to_datetime(df[dt])
    df.index = df["CLINICAL_EVENT_DATETIME"]
    df_monthly = df.resample("MS").count()[["EVENT_ID"]]
    df_monthly.columns = ["Actual"]

    # remove current month
    df_monthly = df_monthly[df_monthly.index < datetime.now().strftime('%Y-%m-01')]
    return df_monthly

def arima_model(df):
    auto_mod = auto_arima(df["Actual"], suppress_warnings=True, seasonal=False, stepwise=False, error_action="ignore")
    model_arima = ARIMA(df["Actual"], order=auto_mod.order)
    fit_arima = model_arima.fit()
    return fit_arima

def make_preds_df(fit, df, n=12):
    fc, se, conf = fit.forecast(n, alpha=0.05)  # 95% conf
    idx = pd.date_range(df.index[-1] + pd.DateOffset(months=1), periods = n, freq="MS")
    fc_series = pd.Series(fc, index=idx)
    lower_series = pd.Series(conf[:, 0], index=idx)
    upper_series = pd.Series(conf[:, 1], index=idx)
    
    df_comb = pd.concat([fc_series.round(0), lower_series.round(0), upper_series.round(0)], axis=1)
    df_comb.index.name = "CLINICAL_EVENT_DATETIME"
    df_comb.columns = ["Forecast", "Lower", "Upper"]
    return df_comb

In [3]:
def add_forecast_slide(p, df, med):
    blank_slide_layout = p.slide_layouts[6]
    slide = p.slides.add_slide(blank_slide_layout)

    chart_data = CategoryChartData()
    chart_data.categories = df.index
    chart_data.add_series("Actual", df["Actual"])
    chart_data.add_series("Forecast", df["Forecast"])

    x, y, cx, cy = Inches(1), Inches(1), Inches(8), Inches(6)
    chart = slide.shapes.add_chart(XL_CHART_TYPE.LINE, x, y, cx, cy, chart_data).chart
    chart = format_graph(chart, med)
    return p

def format_graph(chart, med, font_nm = None, ax_bright = 0.35):
    category_axis = format_axis_title(chart.category_axis, "Month", font_nm, ax_bright)
    category_axis = format_axis_tick_labels(category_axis, font_nm, ax_bright, "mmm yy")
    category_axis = format_axis(category_axis, ax_bright)
#     category_axis.maximum_scale = len(df_combined) + 1
#     df_combined.asfreq("MS").index[-1] + pd.DateOffset(months=1)

    value_axis = format_axis_title(chart.value_axis, "Number of doses", font_nm, ax_bright)
    value_axis = format_axis_tick_labels(value_axis, font_nm, ax_bright, "#,##0")
    value_axis = format_axis(value_axis, ax_bright)

    if med == "ivig":
        med_title = med.upper()
    elif med == "acetaminophen":
        med_title = med.title() + " IV "
    else:
        med_title = med.title()
    
    chart.has_title = True
    chart.chart_title.text_frame.text = med_title + " forecast"
    chart.chart_title.text_frame.paragraphs[0].font.name = font_nm
    chart.chart_title.text_frame.paragraphs[0].font.size = Pt(24)
    chart.chart_title.text_frame.paragraphs[0].font.bold = False
    chart.chart_title.text_frame.paragraphs[0].font.color.theme_color = MSO_THEME_COLOR.TEXT_1
    chart.chart_title.text_frame.paragraphs[0].font.color.brightness = 0.25

    chart.has_legend = False
    # chart.legend.include_in_layout = False
    # chart.legend.position = XL_LEGEND_POSITION.TOP

    # format Actual line
    col_actual = MSO_THEME_COLOR.ACCENT_1
    chart.series[0].format.line.width = Pt(3.5)
    chart.series[0].format.line.color.theme_color = col_actual

    i = len(chart.series[0].points) - 13
    chart = format_marker(chart, 0, i, font_nm, MSO_THEME_COLOR.BACKGROUND_1, col_actual)

    # format Forecast line
    brght_forecast = 0.4

    chart.series[1].format.line.dash_style = MSO_LINE_DASH_STYLE.DASH
    chart.series[1].format.line.width = Pt(2.5)
    chart.series[1].format.line.color.theme_color = col_actual
    chart.series[1].format.line.color.brightness = brght_forecast

    j = len(chart.series[1].points) - 1
    chart = format_marker(chart, 1, j, font_nm, col_actual, col_actual, brght_forecast)
    
    return chart

def format_axis(axis, ax_bright):
    axis.format.line.color.theme_color = MSO_THEME_COLOR.TEXT_1
    axis.format.line.color.brightness = ax_bright
    axis.has_major_gridlines = False
    axis.has_minor_gridlines = False
    return axis

def format_axis_title(axis, title_text, font_nm, ax_bright, has_title = True):
    axis.has_title = has_title
    axis.axis_title.text_frame.text = title_text
    axis.axis_title.text_frame.paragraphs[0].font.name = font_nm
    axis.axis_title.text_frame.paragraphs[0].font.size = Pt(18)
    axis.axis_title.text_frame.paragraphs[0].font.bold = False
    axis.axis_title.text_frame.paragraphs[0].font.color.theme_color = MSO_THEME_COLOR.TEXT_1
    axis.axis_title.text_frame.paragraphs[0].font.color.brightness = ax_bright
    return axis

def format_axis_tick_labels(axis, font_nm, ax_bright, num_fmt):
    axis.tick_labels.font.name = font_nm
    axis.tick_labels.font.size = Pt(16)
    axis.tick_labels.font_bold = False
    axis.tick_labels.font.color.theme_color = MSO_THEME_COLOR.TEXT_1
    axis.tick_labels.font.color.brightness = ax_bright
    axis.tick_labels.number_format = num_fmt
    return axis

def format_marker(chart, i, j, font_nm, col_mark, col_line, brght=0):
    if i == 0:
        mrk_sz = 7
    else: 
        mrk_sz = 5
    
#     chart.series[i].data_labels.show_value = True
    chart.series[i].data_labels.number_format = "#,##0"
    chart.series[i].data_labels.font.name = font_nm
    chart.series[i].data_labels.font.size = Pt(18)
    chart.series[i].data_labels.font.bold = False
    chart.series[i].data_labels.position = XL_DATA_LABEL_POSITION.BELOW
    chart.series[i].points[j].marker.style = XL_MARKER_STYLE.CIRCLE
    chart.series[i].points[j].marker.size = mrk_sz
    chart.series[i].points[j].marker.format.fill.solid()
    chart.series[i].points[j].marker.format.fill.fore_color.theme_color = col_mark
    chart.series[i].points[j].marker.format.fill.fore_color.brightness = brght
    chart.series[i].points[j].marker.format.line.width = Pt(2.5)
    chart.series[i].points[j].marker.format.line.color.theme_color = col_line
    chart.series[i].points[j].marker.format.line.color.brightness = brght

    return chart

In [4]:
def make_slides(meds, n=12):
    prs = Presentation()
    # title slide
    title_slide_layout = prs.slide_layouts[0]
    slide = prs.slides.add_slide(title_slide_layout)
    title = slide.shapes.title
    subtitle = slide.placeholders[1]
    title.text = "Forecast for Target Medications"
    fcast_start = datetime.now().strftime('%B %Y')
    fcast_end = (datetime.now() + pd.DateOffset(months=11)).strftime('%B %Y')
    subtitle.text = "Forecast for: " + fcast_start + " to " + fcast_end + "\nBrian Gulbis, PharmD, BCPS"

    for i in meds:
        if i == "bupivacaine-liposme":
            df_monthly = get_actual_data(i, "orders", "ORDER_DATETIME")
        else:
            df_monthly = get_actual_data(i)
            
        fit_arima = arima_model(df_monthly)
        df_forecast = make_preds_df(fit_arima, df_monthly)
        df_combined = pd.concat([df_monthly, df_forecast], axis=1).replace({pd.np.nan: None})
        add_forecast_slide(prs, df_combined, i)

    return prs

In [12]:
# meds = ["acetaminophen", "albumin", "bupivacaine-liposme", "calcitonin", "eculizumab", "isoproterenol", "ivig", 
#         "pegfilgrastim"]

meds = ["acetaminophen", "albumin", "bupivacaine-liposome", "calcitonin", "isoproterenol", "ivig"]
prs = make_slides(meds)
prs.save('../doc/forecast.pptx')

/home/brian/miniconda3/envs/med_tracking/lib/python3.7/site-packages/statsmodels/base/model.py:492: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)


['../data/tidy/bupivacaine-liposome/bupivacaine-liposome_orders_2016-07.csv',
 '../data/tidy/bupivacaine-liposome/bupivacaine-liposome_orders_2017-01.csv',
 '../data/tidy/bupivacaine-liposome/bupivacaine-liposome_orders_2017-07.csv',
 '../data/tidy/bupivacaine-liposome/bupivacaine-liposome_orders_2018-01.csv',
 '../data/tidy/bupivacaine-liposome/bupivacaine-liposome_orders_2018-07.csv',
 '../data/tidy/bupivacaine-liposome/bupivacaine-liposome_orders_2018-08.csv',
 '../data/tidy/bupivacaine-liposome/bupivacaine-liposome_orders_2018-09.csv',
 '../data/tidy/bupivacaine-liposome/bupivacaine-liposome_orders_2018-10.csv',
 '../data/tidy/bupivacaine-liposome/bupivacaine-liposome_orders_2018-11.csv',
 '../data/tidy/bupivacaine-liposome/bupivacaine-liposome_orders_2018-12.csv',
 '../data/tidy/bupivacaine-liposome/bupivacaine-liposome_orders_2019-01-01.csv',
 '../data/tidy/bupivacaine-liposome/bupivacaine-liposome_orders_2019-01-16.csv',
 '../data/tidy/bupivacaine-liposome/bupivacaine-liposome_o